# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/autos.csv')

Посмотрим на предоставленные данные

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

Теперь займемся предобработкой данных

In [5]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Проверим, есть ли объявления, в которых укзана нулевая стоимость

In [6]:
print("Колличество объявлений с нулевой ценой:",len(data.loc[data['Price'] == 0]))

Колличество объявлений с нулевой ценой: 10772


Удалим такие записи, так как восстановить стоимость невозможно, но это влияет на предсказания моделей

In [7]:
data = data.loc[data['Price'] != 0]

Удалим данные с пропусками в столбце `Model`, так как невозможно заполнить их исходя из основных данных

In [8]:
data = data.loc[~data['Model'].isna()]

Пропусков в столбце `VehicleType` слишком много, поэтому не будем их удалять, а просто заменим на `unknown`

In [9]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')

In [10]:
data['RegistrationYear'].unique()
print(len(data.loc[(data['RegistrationYear'] > 2021) | (data['RegistrationYear'] < 1900)]))

86


В столбце `RegistrationYear` есть некорректные записи, можно удалить их, так как их количество незначительно

In [11]:
data = data.loc[(data['RegistrationYear'] <= 2021) & (data['RegistrationYear'] >= 1900)]

Заменим пропуски в столбце `Gearbox` средним по модели

In [12]:
data['Gearbox'] = data['Gearbox'].fillna(data.groupby('Model')['Gearbox'].transform(lambda x: x.value_counts().idxmax()))

Проверим значения в столбце `Power`

In [13]:
data['Power'].unique()

array([    0,   163,    75,    69,   102,   109,   125,   101,   105,
         140,   115,   131,   190,    60,   136,   160,   231,    90,
          50,   118,   193,    99,   113,   218,   122,   129,    70,
         306,    95,    61,   177,    80,   170,    55,   143,    64,
         232,   150,   156,    82,   155,   185,    87,   180,    86,
          84,   224,   235,   200,   178,   265,    77,   110,   144,
         120,    54,   286,   116,   184,   126,   204,    88,   194,
         305,   197,   179,   250,   313,    41,    45,   165,    98,
         130,   114,   211,    56,   201,   213,    58,   107,    83,
         174,   100,   220,    85,    73,   192,    66,   299,    74,
          52,   147,    65,   310,    71,    97,   239,    68,   203,
         300,   103,   258,   320,    63,    81,   148,    44,   145,
         280,   260,   104,   188,   333,   117,   141,    59,   132,
         234,   158,    39,    92,    51,   135,   230,    53,   209,
          43,   146,

Удалим некорректные значения в этом столбце

In [14]:
data.loc[(data['Power'] > 1000) | (data['Power'] <= 4), 'Power'] = None
data['Power'] = data['Power'].fillna(data.groupby('Model')['Power'].transform('median'))
data = data.loc[~data['Power'].isna()]
data['Power'] = data['Power'].astype('int64')

Пропуск `FuelType` заполним средним по модели

In [15]:
data['FuelType'] = data['FuelType'].fillna(data.groupby('Model')['FuelType'].transform(lambda x: x.value_counts().idxmax()))

Пропуски в столбце `Repaired` заполним значением `no`, так как если машина не ремонтировалась, то пользователь вероятно пропускает этот столбец 

In [16]:
data["Repaired"].fillna("no", inplace=True)

Так же удалим признаки, не влияющии на стоимость

In [17]:
data = data.drop(['NumberOfPictures', 'PostalCode', 'DateCrawled', 'LastSeen', 'DateCreated'], axis=1)

Удалим дубликаты

In [18]:
data = data.drop_duplicates()

Так же удалим признаки, не влияющии на стоимость

In [20]:
data = data.reset_index(drop=True)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294893 entries, 0 to 294892
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              294893 non-null  int64 
 1   VehicleType        294893 non-null  object
 2   RegistrationYear   294893 non-null  int64 
 3   Gearbox            294893 non-null  object
 4   Power              294893 non-null  int64 
 5   Model              294893 non-null  object
 6   Kilometer          294893 non-null  int64 
 7   RegistrationMonth  294893 non-null  int64 
 8   FuelType           294893 non-null  object
 9   Brand              294893 non-null  object
 10  Repaired           294893 non-null  object
dtypes: int64(5), object(6)
memory usage: 24.7+ MB


In [22]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,294893.000000,294893.000000,294893.000000,294893.000000,294893.000000
mean,4626.653379,2003.241623,119.207655,128331.055671,5.832977
std,4547.079506,7.148876,53.260040,37189.848967,3.664232
min,1.000000,1910.000000,5.000000,5000.000000,0.000000
25%,1200.000000,1999.000000,75.000000,125000.000000,3.000000
50%,2950.000000,2003.000000,110.000000,150000.000000,6.000000
75%,6650.000000,2008.000000,144.000000,150000.000000,9.000000
max,20000.000000,2019.000000,1000.000000,150000.000000,12.000000


Перед обучением моделей проведем кодирование категориальных данных с помощью Ordinal Encoding

In [23]:
old_data = data

In [24]:
target = data['Price']
features = data.drop('Price', axis=1)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)
cat_features = ['VehicleType', 'Repaired', 'FuelType', 'Gearbox', 'Brand', 'Model']

In [25]:
ohe_encoder = OneHotEncoder(drop='first', sparse=False)

train_temp = ohe_encoder.fit_transform(features_train[cat_features])
features_train[ohe_encoder.get_feature_names()] = train_temp
features_train.drop(cat_features, axis=1, inplace=True)

test_temp = ohe_encoder.transform(features_test[cat_features])
features_test[ohe_encoder.get_feature_names()] = test_temp
features_test.drop(cat_features, axis=1, inplace=True)

Стандартизируем данные

In [27]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_test = scaler.transform(features_test)

**Вывод: Был обработан предоставленный массив данных. Заполненны пропуски в таблице, исправленны некорректные данные. Удалены дубликаты. Так же удалены столбцы с данными не влияющими на стоимость. Проведено кодирование данных методом `Ordinal Encoding`. Так же проведено масштабирование данных.**

## Обучение моделей

### DecisionTreeRegressor

In [28]:
test_tree_model = DecisionTreeRegressor(criterion='mse', random_state=12345) 
test_tree_params = {'max_depth': list(np.arange(4, 12))}
test_tree_model = RandomizedSearchCV(test_tree_model,  test_tree_params, n_iter=100, n_jobs=-1, cv=2, verbose=10)
test_tree_model.fit(features_train, target_train)
test_tree_model.best_params_

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2; 1/8] START max_depth=4.................................................
[CV 1/2; 1/8] END ...............................max_depth=4; total time=   3.2s
[CV 2/2; 1/8] START max_depth=4.................................................
[CV 2/2; 1/8] END ...............................max_depth=4; total time=   3.0s
[CV 1/2; 2/8] START max_depth=5.................................................
[CV 1/2; 2/8] END ...............................max_depth=5; total time=   3.7s
[CV 2/2; 2/8] START max_depth=5.................................................
[CV 2/2; 2/8] END ...............................max_depth=5; total time=   3.7s
[CV 1/2; 3/8] START max_depth=6.................................................
[CV 1/2; 3/8] END ...............................max_depth=6; total time=   3.9s
[CV 2/2; 3/8] START max_depth=6.................................................
[CV 2/2; 3/8] END ...............................

{'max_depth': 11}

In [29]:
%%time
tree_model = DecisionTreeRegressor(criterion='mse', max_depth=11, random_state=12345) 
tree_model.fit(features_train, target_train)

CPU times: user 5.81 s, sys: 88.9 ms, total: 5.9 s
Wall time: 6.07 s


DecisionTreeRegressor(max_depth=11, random_state=12345)

In [30]:
%%time
pred_tree_model = tree_model.predict(features_test)

CPU times: user 48.2 ms, sys: 30.8 ms, total: 79 ms
Wall time: 87.8 ms


In [31]:
mse_tree_model = mean_squared_error(target_test, pred_tree_model)
print("RMSE для DecisionTreeRegressor модели на тестовой выборке:", round((mse_tree_model) ** 0.5, 2))

RMSE для DecisionTreeRegressor модели на тестовой выборке: 1976.05


### CatBoostRegressor

In [32]:
old_target = old_data['Price']
old_features = old_data.drop('Price', axis=1)
old_features_train, old_features_test, old_target_train, old_target_test = train_test_split(
    old_features, old_target, test_size=0.25, random_state=12345)

In [33]:
%%time
cat_model = CatBoostRegressor(loss_function='RMSE', n_estimators=500 ,learning_rate=0.5, random_state=12345, verbose=True) 
cat_model.fit(old_features_train, old_target_train, cat_features=cat_features)

0:	learn: 3235.7913801	total: 472ms	remaining: 3m 55s
1:	learn: 2638.5487782	total: 821ms	remaining: 3m 24s
2:	learn: 2340.1527015	total: 1.14s	remaining: 3m 9s
3:	learn: 2186.2649149	total: 1.42s	remaining: 2m 55s
4:	learn: 2107.4663226	total: 1.64s	remaining: 2m 42s
5:	learn: 2056.4534747	total: 1.88s	remaining: 2m 34s
6:	learn: 2022.4220249	total: 2.12s	remaining: 2m 29s
7:	learn: 1990.6659814	total: 2.33s	remaining: 2m 23s
8:	learn: 1957.7234608	total: 2.62s	remaining: 2m 23s
9:	learn: 1941.9575127	total: 2.89s	remaining: 2m 21s
10:	learn: 1926.5241567	total: 3.07s	remaining: 2m 16s
11:	learn: 1912.6232643	total: 3.24s	remaining: 2m 11s
12:	learn: 1900.9284195	total: 3.42s	remaining: 2m 8s
13:	learn: 1893.1792659	total: 3.66s	remaining: 2m 6s
14:	learn: 1881.9460829	total: 3.91s	remaining: 2m 6s
15:	learn: 1876.0745401	total: 4.12s	remaining: 2m 4s
16:	learn: 1867.2925891	total: 4.28s	remaining: 2m 1s
17:	learn: 1857.1206405	total: 4.46s	remaining: 1m 59s
18:	learn: 1848.2843384	to

In [34]:
%%time

cat_predict = cat_model.predict(old_features_test)

CPU times: user 524 ms, sys: 3.25 ms, total: 528 ms
Wall time: 556 ms


In [35]:
mse_cat_model = mean_squared_error(target_test, cat_predict)
print("RMSE для CatBoostRegressor модели на тестовой выборке:", round((mse_cat_model) ** 0.5, 2))

RMSE для CatBoostRegressor модели на тестовой выборке: 1622.75


### LGBMRegressor

In [36]:
%%time

lgbm_model = LGBMRegressor(learning_rate=0.1, num_leaves=100, random_state=12345)
lgbm_model.fit(features_train, target_train)

CPU times: user 20min 17s, sys: 14.9 s, total: 20min 31s
Wall time: 20min 38s


LGBMRegressor(num_leaves=100, random_state=12345)

In [37]:
%%time

lgbm_predict = lgbm_model.predict(features_test)

CPU times: user 1.15 s, sys: 34.7 ms, total: 1.19 s
Wall time: 1.18 s


In [38]:
mse_lgbm_model = mean_squared_error(target_test, lgbm_predict)
print("RMSE для LGBMRegressor модели на тестовой выборке:", round((mse_lgbm_model) ** 0.5, 2))

RMSE для LGBMRegressor модели на тестовой выборке: 1632.16


**Вывод: Для решения задачи предложенно 4 модели: `LinearRegression`, ` DecisionTreeRegressor`, `CatBoostRegressor` и ` LGBMRegressor`. Было посчитанно время работы каждой модели и полученно значение метрики RMSE для каждой модели.**

## Анализ моделей

In [40]:
inf = [["6.07 s", "87.8 ms", 1976.05],
       ["1min 50s", "556 ms", 1622.75],
       ["20min 38 s", "1.18 s", 1632.16]]
models = ["DecisionTreeRegressor", "CatBoostRegressor", "LGBMRegressor"]
pd.DataFrame(data=inf, index=models, columns=["fit_time", "predict_time", "RMSE"])

,fit_time,predict_time,RMSE
DecisionTreeRegressor,6.07 s,87.8 ms,1976.05
CatBoostRegressor,1min 50s,556 ms,1622.75
LGBMRegressor,20min 38 s,1.18 s,1632.16


**Вывод: Лучшее значение метрики полученно с помощью `CatBoostRegressor` $(RMSE = 1622.75)$. Модель `LGBMRegressor` является самой медленной (время обучения 20 min 38 s). Самой оптимальной моделью является `CatBoostRegressor`: время работы данного метода (1min 50s) не очень большое, хоть и значительно превосходит `DecisionTreeRegressor`, но точность `DecisionTreeRegressor` отличается значительно ($RMSE = 1976.05$ вместо $1622.75$).**

## Общий вывод

**В процессе работы была выполнена предобработка данных. А именно: удалены объявления с нулевой стоимостью, записи с пропусками в столбце `Model`, записи с некорректными значениями `RegistrationYear` и `Power`. Заполненны пропуски в столбце `Repaired`. Удалены столбцы, не влияющие на стоимость `NumberOfPictures`, `PostalCode`, `DateCrawled`, `LastSeen`.
Была проведено кодирование категориальных данных с помощью Ordinal Encoding, так же данные были стандартизованны.
Было обученно четыре модели ` DecisionTreeRegressor`, `CatBoostRegressor` и `LGBMRegressor`.
В ходе анализа полученных результатов, было выясненно что оптимальной моделью для данной задачи является `CatBoostRegressor` ($RMSE = 1622.75, t_{work}$ = 1min  50s)**